In [1]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
import scanpy as sc 
import sys 
import umap
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm import tqdm
import glob

sys.path.append('../../src')

from spaceoracle import SpaceTravLR

from spaceoracle.tools.network import RegulatoryFactory
from spaceoracle.gene_factory import GeneFactory

In [3]:
base_dir = '/Users/koush/Desktop/'

co_grn = RegulatoryFactory(
    colinks_path=base_dir + 'training_data/snrna_human_tonsil_colinks.pkl',
    annot='cell_type_int'
)

adata = sc.read_h5ad(base_dir + 'training_data/snrna_human_tonsil.h5ad')
cell_threshes = pd.read_parquet(base_dir + 'training_data/tonsil_LRs.parquet')
adata.uns['cell_thresholds'] = cell_threshes

In [4]:
gf = GeneFactory.from_json(
    adata, 
    '/Volumes/SSD/human_tonsil/run_params.json', 
    override_params={'save_dir': '/Volumes/SSD/human_tonsil'}
)

In [5]:
gf.load_betas(subsample=10, float16=True)

In [6]:
gex_out = gf.perturb(
    target=['GATA3', 'IL4'],
    n_propagation=0,
    gene_expr=[0, 2],
)

GATA3_0n_0x_IL4_0n_2x


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
